# County Covid Data in the United States

In this notebook, we well be pulling in coronavirus data from The New York Times's github. The data are provided at a county-level resolution, with only deaths and cases provided for each county.

In this notebook, we will assess whats available, the accuracy of the data, and the potential for processing to make use of these data in visualizations.

---

## 1. Notebook Setup

First, we need to make sure we are in the root directory of the project to import custom modules. These notebooks are stored in `notebooks/` for cleanliness.

In [1]:
pwd

'/Users/DanOvadia/Projects/covid-hotspots/notebooks'

In [2]:
cd ..

/Users/DanOvadia/Projects/covid-hotspots


### Import Libraries

In [3]:
# To hold and manipulate data
import pandas as pd

# To check across all dates for data accuracy
from datetime import date, timedelta

# To track times
import time

### Import Custom Modules

In [4]:
from modules import data_processing
from modules import plotting # not needed, but available

# Extension to auto reload custom modules
%load_ext autoreload

%autoreload 1

%aimport modules.data_processing
%aimport modules.plotting

---

## 2. Data Importing

### County Coronavirus Data
We import data from [New York Times GitHub](https://github.com/nytimes/covid-19-data) to get county level coronavirus data.

In [26]:
URL = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

COVID_COUNTIES_DF = pd.read_csv(URL)

In [27]:
COVID_COUNTIES_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495663 entries, 0 to 495662
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    495663 non-null  object 
 1   county  495663 non-null  object 
 2   state   495663 non-null  object 
 3   fips    490889 non-null  float64
 4   cases   495663 non-null  int64  
 5   deaths  495663 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 22.7+ MB


---

## 3 - Cleaning

We will need to change `date` to datetime, and change `fips` to a string of length 5.

Since the fips-code will be our link to the geojson in choropleth graphs, we will need to check if we are missing any and how we can potentially match them manually.

In [28]:
# Set date format
COVID_COUNTIES_DF['date'] = pd.to_datetime(COVID_COUNTIES_DF['date'], format = '%Y-%m-%d')

# Reassign our fips to be a string of length 5
COVID_COUNTIES_DF['fips'] = COVID_COUNTIES_DF['fips'].astype(str).apply(lambda x: '0'+x[:4] if len(x) == 6 else x[:5])

In [29]:
COVID_COUNTIES_DF = COVID_COUNTIES_DF.merge(data_processing.get_census_county_data(),
                how='left',
                left_on='fips',
                right_on='FIPS')

In [30]:
COVID_COUNTIES_DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495663 entries, 0 to 495662
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   date             495663 non-null  datetime64[ns]
 1   county           495663 non-null  object        
 2   state            495663 non-null  object        
 3   fips             495663 non-null  object        
 4   cases            495663 non-null  int64         
 5   deaths           495663 non-null  int64         
 6   FIPS             480936 non-null  object        
 7   STATE            480936 non-null  float64       
 8   COUNTY           480936 non-null  float64       
 9   POPESTIMATE2019  480936 non-null  float64       
 10  CENSUS2010POP    480936 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 45.4+ MB


In [31]:
COVID_COUNTIES_DF['cases']/COVID_COUNTIES_DF['POPESTIMATE2019']

0         1.216422e-06
1         1.216422e-06
2         1.216422e-06
3         1.941660e-07
4         1.216422e-06
              ...     
495658    7.179463e-03
495659    1.845380e-02
495660    1.498072e-02
495661    1.383728e-02
495662    2.742890e-03
Length: 495663, dtype: float64

In [32]:
# Cases Per Million
COVID_COUNTIES_DF['casesPerMillion']=COVID_COUNTIES_DF['cases']/COVID_COUNTIES_DF['POPESTIMATE2019']*1000000

In [34]:
COVID_COUNTIES_DF['case_diff'] = COVID_COUNTIES_DF.groupby(
        by = ['fips','county','state'])['cases'].diff()

In [49]:
COVID_COUNTIES_DF['cases_14MA'] = COVID_COUNTIES_DF.groupby(
    by=['fips','county','state'], 
    as_index=False)['case_diff'].rolling(14).mean().reset_index(level=[0,1,2], drop=True)

In [48]:
a

9480            NaN
10835           NaN
12367           NaN
14025           NaN
15803           NaN
            ...    
495629    26.285714
43197           NaN
45795           NaN
56455           NaN
59163           NaN
Name: case_diff, Length: 495663, dtype: float64

---

## 4 - Checks

### Checking consistency across dates

- Check if any county has holes in their reporting. Make sure dates are consistent from first reporting date to today.

- Create a list of counties that have never reported.

- Create a list of counties that have errors with their fips codes, find them and repiar them.

In [16]:
# Check for nulls
COVID_COUNTIES_DF['fips'].isna().sum()

0

That's odd. I know there are nulls in here, since NYC is included as one county. I think they just record it as a string. Lets check, NYC.

In [27]:
COVID_COUNTIES_DF[COVID_COUNTIES_DF['county'] == 'New York City'].head(2)

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,nan,1,0
448,2020-03-02,New York City,New York,nan,1,0


In [28]:
# Check the type of fips for NYC and return a sample
print(type(COVID_COUNTIES_DF[COVID_COUNTIES_DF['county'] == 'New York City']['fips'].iloc[0]))
COVID_COUNTIES_DF[COVID_COUNTIES_DF['county'] == 'New York City']['fips'].iloc[0]

<class 'str'>


'nan'

In [14]:
# Check the number of "nan" strings in covid counties.
(COVID_COUNTIES_DF['fips'] == 'nan').sum()

4743

That seems like a lot of nulls, but remember this dataset has almost half a million rows, and there are 3,141 counties in the United States.

In [32]:
# Number of days since March 1st.
len(pd.date_range(start='2020-03-01',end='2020-09-01'))

185

The pandemic has also been raging in the United States for around 185 days.

Lets get a list of (unique county), state pairs that have null fips.

In [108]:
# Create a mask of counties with fips = 'nan'
NAN_COUNTY_MASK = (COVID_COUNTIES_DF['fips'] == 'nan')
DATE_MASK = (COVID_COUNTIES_DF['date'] == '2020-09-01')

# Subset our data and only grab county/state names
NAN_COUNTY_SERIES = COVID_COUNTIES_DF[nan_mask].apply(lambda x: (str(x['county']), str(x['state'])), axis = 1)

# Grab a unique set of county, state tuples
NAN_UNIQUE_COUNTIES = list(pd.unique(NAN_SERIES))

# Return the length of our list
len(NAN_UNIQUE_COUNTIES)

54

Lets see how many cases and deaths are coming from these nan counties (Specifically as of today)

In [109]:
print(f"Cases: {COVID_COUNTIES_DF[NAN_COUNTY_MASK & DATE_MASK]['cases'].sum()}")
print(f"Deaths: {COVID_COUNTIES_DF[NAN_COUNTY_MASK & DATE_MASK]['deaths'].sum()}")

Cases: 269322
Deaths: 24328


Most of these cases are definitely coming from New York. If we fix the non-Unknown values. That will likely make a big difference in our data accuracy. We can also examine to see how many cases are in the unknown counties separately.

In [58]:
NAN_UNIQUE_COUNTIES

[('New York City', 'New York'),
 ('Unknown', 'Rhode Island'),
 ('Unknown', 'New Jersey'),
 ('Unknown', 'Puerto Rico'),
 ('Unknown', 'Virgin Islands'),
 ('Unknown', 'Guam'),
 ('Unknown', 'Maine'),
 ('Unknown', 'Massachusetts'),
 ('Unknown', 'Louisiana'),
 ('Unknown', 'Kentucky'),
 ('Unknown', 'Nevada'),
 ('Unknown', 'Tennessee'),
 ('Unknown', 'Arkansas'),
 ('Unknown', 'Georgia'),
 ('Kansas City', 'Missouri'),
 ('Unknown', 'Missouri'),
 ('Unknown', 'Minnesota'),
 ('Unknown', 'California'),
 ('Unknown', 'Colorado'),
 ('Unknown', 'Florida'),
 ('Unknown', 'Hawaii'),
 ('Unknown', 'Illinois'),
 ('Unknown', 'Vermont'),
 ('Unknown', 'Arizona'),
 ('Unknown', 'Michigan'),
 ('Unknown', 'Texas'),
 ('Unknown', 'Virginia'),
 ('Unknown', 'Washington'),
 ('Unknown', 'Utah'),
 ('Unknown', 'Idaho'),
 ('Unknown', 'Mississippi'),
 ('Unknown', 'Northern Mariana Islands'),
 ('Unknown', 'New York'),
 ('Unknown', 'Connecticut'),
 ('Unknown', 'Nebraska'),
 ('Unknown', 'Montana'),
 ('Unknown', 'Pennsylvania'),
 

These unknown county names could very well represent multiple counties. Will have to dig deeper later to determine if they are unique or multiple.

#### Investigating California's Unknown county

Lets take one example and see if this is just an error with one county. Or if this represents something broadly more wrong with these data.

In [116]:
def check_state_nans(df, state):
    state_mask = (df['state'] == state)
    county_mask = (df['county'] == 'Unknown')
    state_nans = df[state_mask & county_mask]
    print(f"{state}")
    print(f"{len(state_nans)} county dates")
    print(f"Average of {state_nans['cases'].mean()} cases")
    print(f"Average of {state_nans['deaths'].mean()} deaths")
    
#for state in 
check_state_nans(COVID_COUNTIES_DF, 'Kansas')

Kansas
16 county dates
Average of 0.0 cases
Average of 2.4375 deaths


In [94]:
a = list(pd.unique(COVID_COUNTIES_DF[STATE_MASK]['county']))

In [95]:
a.sort()

In [97]:
len(a)

96

In [82]:
df = pd.read_csv('data/census_county_est2019.csv')

In [84]:
state_mask = (df['STNAME'] == 'Tennessee')

In [89]:
state_mask.sum()

96

In [88]:
df[state_mask].sort_values(by = 'CTYNAME')

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
2471,50,3,6,47,1,Tennessee,Anderson County,75129,75082,75098,...,10.217597,2.448322,2.113884,2.404932,0.611474,5.829510,3.046680,10.001056,5.592643,10.883111
2472,50,3,6,47,3,Tennessee,Bedford County,45058,45057,45078,...,5.725326,-0.132854,-3.734765,3.479448,11.370567,11.974377,7.988134,13.381703,15.181240,7.505639
2473,50,3,6,47,5,Tennessee,Benton County,16489,16491,16511,...,0.370302,3.874092,1.395039,-2.011888,-2.887244,6.663171,-3.033868,3.055339,21.836906,0.432019
2474,50,3,6,47,7,Tennessee,Bledsoe County,12876,12874,12884,...,13.089792,8.424795,-5.555984,71.545685,41.040442,6.941104,9.407274,12.891034,1.410437,14.559054
2475,50,3,6,47,9,Tennessee,Blount County,123010,123098,123199,...,13.932433,3.937407,2.777688,7.521153,8.216304,9.224647,11.096396,14.992469,11.671067,14.469459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,50,3,6,47,181,Tennessee,Wayne County,17021,17025,16985,...,6.605020,4.060615,-1.530627,1.003157,-0.118413,-0.535284,2.864475,-0.958428,3.305686,6.665065
2562,50,3,6,47,183,Tennessee,Weakley County,35021,35015,35044,...,-2.037728,-4.463455,-8.759419,-9.460187,-5.575445,-2.858279,-6.816633,-5.946067,4.315383,-1.018864
2563,50,3,6,47,185,Tennessee,White County,25841,25844,25828,...,12.125443,10.640349,1.881684,7.850797,5.680843,4.714291,7.764416,13.714844,15.264624,12.309162
2564,50,3,6,47,187,Tennessee,Williamson County,183182,183277,184143,...,20.557410,18.124537,19.024815,24.646110,25.123802,24.574338,28.970426,27.306378,21.369966,22.997938


In [90]:
STATE_MASK = (COVID_COUNTIES_DF['state'] == 'Tennessee')
COUNTY_MASK = (COVID_COUNTIES_DF['county'] == 'Unknown')

CA_NAN = COVID_COUNTIES_DF[STATE_MASK & COUNTY_MASK]

In [91]:
CA_NAN.sort_values(by='date', ascending = True)

,date,county,state,fips,cases,deaths
5038,2020-03-19,Unknown,Tennessee,nan,2,0
5927,2020-03-20,Unknown,Tennessee,nan,2,0
6944,2020-03-21,Unknown,Tennessee,nan,8,0
8074,2020-03-22,Unknown,Tennessee,nan,28,0
9292,2020-03-23,Unknown,Tennessee,nan,109,0
...,...,...,...,...,...,...
478863,2020-08-28,Unknown,Tennessee,nan,206,0
482097,2020-08-29,Unknown,Tennessee,nan,146,5
485332,2020-08-30,Unknown,Tennessee,nan,102,5
488565,2020-08-31,Unknown,Tennessee,nan,43,5


In [65]:
len(CA_NAN)

30

First of all the entries are not consecutive; there are gaps. Second, I would expect cases and deaths to be cumulative and therefore move in a positive direction. That is not the case here. Besides, only 9 unknown rows for California?

These data could very well be coming from multiple counties or mistakes along the reporting pipeline.

In [5]:
COVID_COUNTIES_DF = data_processing.get_covid_county_data(cache_mode = 0)

Retrieving Covid County data
Pulling county data from github.


TypeError: incompatible index of inserted column with frame index

In [13]:
url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv'

# Read in the data to dataframe
census_df = pd.read_csv(url, encoding = "ISO-8859-1")

In [19]:
census_df.index

RangeIndex(start=0, stop=3193, step=1)

In [20]:
census_df[['STATE','COUNTY']].apply(
        lambda x:generate_fips(
            x['STATE'],
            x['COUNTY']
        ), axis=1
    ).index

RangeIndex(start=0, stop=3193, step=1)

In [14]:
def generate_fips(state_fips, county_fips):
    state_str, county_str = str(state_fips), str(county_fips)
    
    # Check length of state code and append 0's if necessary.
    if len(state_str) == 1:
        state_str = "0"+state_str
        
    # Check length of county code and append 0's if necessary.
    if len(county_str) == 1:
        county_str = "00"+county_str
    elif len(county_str) == 2:
        county_str = "0"+county_str
        
    return state_str+county_str

In [21]:
census_df['FIPS'] = census_df[['STATE','COUNTY']].apply(
        lambda x:generate_fips(
            x['STATE'],
            x['COUNTY']
        ), axis=1
    )

In [ ]:
census_df.info(verbose = True)

### Merge Dataframes

To calculate cases and deaths per capita at a county level, we will merge our population reference data with county covid data on fips codes. Reminder: County covid data has many rows of the same fips code since it is a time series. We may need to check for consistency.

In [ ]:
# Merge dataframes on fips
DF = COVID_COUNTIES_DF.merge(CENSUS_COUNTIES_DF, 
                        how='left',
                        left_on='fips',
                        right_on='FIPS')

In [117]:
COVID_COUNTIES_DF

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
492424,2020-09-01,Sweetwater,Wyoming,56037,302,2
492425,2020-09-01,Teton,Wyoming,56039,430,1
492426,2020-09-01,Uinta,Wyoming,56041,301,2
492427,2020-09-01,Washakie,Wyoming,56043,107,6


In [23]:
census_df = data_processing.get_census_county_data()

In [24]:
%%time
# Get county data
x0 = time.time()
COVID_COUNTIES_DF = data_processing.get_covid_county_data()
time.time() - x0

Retrieving Covid County data
Pulling county data from github.


TypeError: incompatible index of inserted column with frame index

In [ ]:
%%time
COVID_COUNTIES_DF.to_csv('data/covid_counties_20200901.csv.gz',
                        index = False,
                        compression = 'gzip')

In [ ]:
%%time
COVID_COUNTIES_2 = pd.read_csv('data/covid_counties_20200901.csv.gz',
                              compression = 'gzip')

In [ ]:
COVID_COUNTIES_DF.info()

In [ ]:
# Take a look at columns for reference and context
COVID_COUNTIES_DF.columns

### County Census Data

We import data from census.gov to get county level population data. Specifically using 2019 estimates.

In [ ]:
# Get Census Data
CENSUS_COUNTIES_DF = data_processing.get_census_county_data()

In [ ]:
# Take a look at columns for reference and context
CENSUS_COUNTIES_DF.columns

## Merge Dataframes

To calculate cases and deaths per capita at a county level, we will merge our population reference data with county covid data on fips codes. Reminder: County covid data has many rows of the same fips code since it is a time series. We may need to check for consistency.

In [ ]:
# Merge dataframes on fips
DF = COVID_COUNTIES_DF.merge(CENSUS_COUNTIES_DF, 
                        how='left',
                        left_on='fips',
                        right_on='FIPS')

In [ ]:
DF.info()

Now lets create the two columns we care about:
 - Cases per million population
 - Deaths per million population

In [ ]:
DF['casesPerMillion']=DF['cases']/DF['POPESTIMATE2019']*1000000

In [ ]:
DF['deathsPerMillion']=DF['deaths']/DF['POPESTIMATE2019']*1000000

In [ ]:
DF.head()

In [ ]:
fips_mask = (DF['fips'] == '01001')
date_mask = DF['date'] == '2020-08-28'

DF[fips_mask & date_mask]

In [ ]:
%%time
COVID_COUNTIES_DF['case_diff2'] = COVID_COUNTIES_DF['cases'].diff()

In [ ]:
%%time
COVID_COUNTIES_DF['case_diff'] = COVID_COUNTIES_DF.sort_values(
    by=['fips','state','county','date'])['cases'].diff()

In [ ]:
%%time
COVID_COUNTIES_DF['cases_diff_group'] = COVID_COUNTIES_DF.groupby(
    by = ['fips','county','state'])['cases'].diff()

In [ ]:
%%time
cases_14MA = COVID_COUNTIES_DF.groupby(
    by = ['fips','county','state'],as_index=False)['cases'].rolling(14).mean()

COVID_COUNTIES_DF["cases_14MA"] = cases_14MA.reset_index(level=0, drop=True)

In [ ]:
type(a)

In [ ]:
print((COVID_COUNTIES_DF['case_diff'] != COVID_COUNTIES_DF['case_diff2']).sum())
print((COVID_COUNTIES_DF['case_diff'] != COVID_COUNTIES_DF['cases_diff_group']).sum())

In [ ]:
disc_mask = (COVID_COUNTIES_DF['cases_diff_group'] != COVID_COUNTIES_DF['case_diff'])
COVID_COUNTIES_DF[disc_mask]

In [ ]:
COVID_COUNTIES_DF[['date','county','state','fips','cases','case_diff','cases_diff_group']]

In [ ]:
df.sort_values(by=['fips','state','county','date'])['cases']

In [ ]:
dfmask = (COVID_COUNTIES_DF['fips'] != COVID_COUNTIES_DF['fips'].shift(1))
COVID_COUNTIES_DF[dfmask]['case_diff'] = np.nan

In [ ]:
def get_county_data_by_dates(df, fips, start_date, end_date):
    fips_mask = (df['fips'] == fips)
    date_mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    return df[fips_mask & date_mask]

get_county_data_by_dates(COVID_COUNTIES_DF,'24015','2020-02-01','2020-04-10')[['date','county','state','cases','deaths','case_diff']]

In [ ]:
negative_mask = (DF['case_diff'] < 0)
DF[negative_mask].sort_values(by=['fips','date'])

In [ ]:
[np.isnan(x) for x in DF['case_diff'].head()]

In [ ]:
plotting.scatter_deaths_county(DF,'case_diff','2020-08-28','06037')

In [ ]:
DF['cases']

### Consistentcy Check

Here I want to check to see if we still have 30 nulls coming from fips codes. 

Since we're dealing with time series I want to check across all available dates to make sure we're consistent across all dates. Or at least that there is some sense as to why the data are missing.

First lets run a consistency check for one date. And create a function to check each specific date.

In [ ]:
def check_null_fips(df, date, seen_set=set()):
    # Create a mask for a specific date.
    date_mask = (df['date'] == date)

    # Create a mask for when fips are null.
    fips_null_mask = (df['fips'].map(lambda x:len(x)) < 5)
    
    nulls_set = set(df[fips_null_mask & date_mask][['county','state']].apply(
        lambda x:str(x['county'])+str(x['state']), axis=1))
    
    # Print if find new
    if len(nulls_set.difference(seen_set.intersection(nulls_set))) != 0:
        print(f"{len(nulls_set.difference(seen_set.intersection(nulls_set)))} NEW nulls. {len(seen_set | nulls_set)} Total Unique Nulls on {date}")

    # Update seen_set
    seen_set = seen_set | nulls_set
    
    return seen_set #df[fips_null_mask & date_mask].shape[0]

# Test it out
a = check_null_fips(DF,'2020-08-28')

Here we see that we have 30 nulls, which is what we expected. Lets check across all dates now.

In [ ]:
# 
sdate = date(2020, 3,1) # start date
edate = date(2020,8,28) # end date

date_list = [date.strftime(format="%Y-%m-%d") 
    for date in 
        pd.date_range(sdate, edate-timedelta(days=1), freq='d')]

In [ ]:
# Create a list of sets seen
seen_set = set()

# Loop over date_list
for date in date_list:
    
    # First empty, then continues to grow
    seen_set = check_null_fips(DF, date, seen_set)

In [ ]:
date_mask = (DF['date'] == '2020-08-28')

In [ ]:
DF[date_mask]['cases']

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(DF[date_mask]['casesPerMillion'])

In [ ]:
date_mask = (COVID_COUNTIES_DF['date'] == '2020-08-28')
category = 'cases'
category_series = COVID_COUNTIES_DF[date_mask][category]
print(f"Mean of {category} is {category_series.mean()}")
print(category_series.quantile([0, 0.1, 0.25, 0.5, 0.75, 0.8,0.85,0.9, 0.95, 0.99,1]))

In [ ]:
DF[date_mask]['case_diff'].max()

In [ ]:
date_mask = (COVID_COUNTIES_DF['date'] == '2020-03-01')

In [ ]:
DF